In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import pandas as pd
import glob
import data_utils.data_utils_celeba as data_utils_celeba


import numpy as np
import pandas as pd
import os

In [2]:
# Root directory for dataset
dataroot = '/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Deep Learning/Projects/Algorithmic fairness/Data/'

# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

In [ ]:
attributes = pd.read_csv(dataroot+'Anno/list_attr_celeba.txt', sep=" ")
attributes.drop('Unnamed: 41',axis=1, inplace=True)
partition = pd.read_csv(dataroot+'Eval/list_eval_partition.txt', sep=" ", header=None, names=['im_id','partition'])
matched = attributes.set_index('im_id').join(partition.set_index('im_id'))

In [ ]:
train = matched[matched['partition']==0]
valid = matched[matched['partition']==1]
test = matched[matched['partition']==2]
train.to_csv('Data/train.csv')
valid.to_csv('Data/valid.csv')
test.to_csv('Data/test.csv')

In [ ]:
len(test)

In [ ]:
val = pd.read_csv("Data/train_full.csv")

In [ ]:
val.columns

In [ ]:
val.groupby("Young").count()
#df = df.groupby('domain')['ID'].nunique()

In [ ]:
IMAGE_SHAPE = (178,218)
# Paths to data
TRAIN_PATH = "Data/train_full.csv" 
VALID_PATH = "Data/valid_full.csv" 
TEST_PATH = "Data/test_full.csv" 
IMAGE_PATHS = glob.glob('/Users/MartinJohnsen/Documents/Martin Johnsen/MMC/3. Semester/Deep Learning/Projects/Algorithmic fairness/Data/celeba/*.jpg')
TARGET_COL = 'Smiling'

# train holds both X (input) and t (target/truth)
data = data_utils_celeba.load_data(train_path=TRAIN_PATH,
                            valid_path = VALID_PATH,
                            test_path=TEST_PATH,
                            image_paths=IMAGE_PATHS,
                            target_col = TARGET_COL,
                            image_shape=IMAGE_SHAPE[:2])

Loading training data
	0 of 162770
	10000 of 162770
	20000 of 162770
	30000 of 162770


In [ ]:
data

In [ ]:
import pickle

## Dump the data into a pickle file
#
#with open('celeba_dataset.pickle', 'wb') as f:
#    pickle.dump(data, f)
    
## Load the data from a pickle file
#with open('Kaggle/data.pickle', 'rb') as f:
#    data = pickle.load(f)

In [ ]:
NUM_CLASSES = 2

dummy_batch_gen = data_utils_celeba.batch_generator(data, batch_size=batch_size, num_classes=NUM_CLASSES, num_iterations=5e3, num_features=40, seed=42)

In [ ]:
train_batch = next(dummy_batch_gen.gen_train())
valid_batch, i = next(dummy_batch_gen.gen_valid())
test_batch, i = next(dummy_batch_gen.gen_test())

In [ ]:
train_batch

In [ ]:
plt.imshow(np.transpose(vutils.make_grid(torch.Tensor(valid_batch['images'][1]), padding=2, normalize=True).cpu(),(0,1,2)))

### Build model:

In [ ]:
# Load functions
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

In [ ]:
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")


def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x


def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

##### Defining the dimensions of the data

In [ ]:
# The image shape should be of the format (height, width, channels)
IMAGE_SHAPE = (178, 218, 3)   # Channel 1 due to B/W images
NUM_CLASSES =    # Different flowers

# For all three features types margin, shape, and texture, we have NUM_FEATURES for each type.
NUM_FEATURES = 64  # <-- Your answer here

##### Defining convolutional dimensions
This formula defines the number of features in both the height and width

In [ ]:
def compute_conv_dim(dim_size):
    return int((dim_size - kernel_size + 2 * conv_pad) / conv_stride + 1)